In [66]:
import pandas as pd

df_trajectories = pd.read_csv('Data/010/trajectories.csv',
                              names=['Latitude', 'Longitude', '0', 'Altitude', 'Date', 'Date_str', 'Time_str'],
                              parse_dates=True)

df_trajectories['DateTime'] = df_trajectories['Date_str'] + ' ' + df_trajectories['Time_str']
df_trajectories['DateTime'] = pd.to_datetime(df_trajectories['DateTime'])

df_trajectories = df_trajectories.drop(columns=['Date_str', 'Time_str', '0', 'Date'])

df_trajectories

,Latitude,Longitude,Altitude,DateTime
0,39.921712,116.472343,13,2007-08-04 03:30:32
1,39.921705,116.472343,13,2007-08-04 03:30:33
2,39.921695,116.472345,13,2007-08-04 03:30:34
3,39.921683,116.472342,13,2007-08-04 03:30:35
4,39.921672,116.472342,13,2007-08-04 03:30:36
...,...,...,...,...
935571,39.136261,117.218261,-59,2009-03-21 05:34:49
935572,39.136256,117.218276,-59,2009-03-21 05:34:50
935573,39.136256,117.218291,-59,2009-03-21 05:34:51
935574,39.136256,117.218303,-59,2009-03-21 05:34:52


In [4]:
df_labels = pd.read_csv('Data/010/labels.txt', delim_whitespace=True, skiprows=1, header=None)
df_labels['StartTime'] = df_labels[0] + ' ' + df_labels[1]
df_labels['EndTime'] = df_labels[2] + ' ' + df_labels[3]
df_labels['StartTime'] = pd.to_datetime(df_labels['StartTime'])
df_labels['EndTime'] = pd.to_datetime(df_labels['EndTime'])
df_labels = df_labels.drop(columns=[0, 1, 2, 3])
df_labels = df_labels.rename(columns={4: "TransportMode"})

df_labels

,TransportMode,StartTime,EndTime
0,bus,2007-06-26 11:32:29,2007-06-26 11:40:29
1,train,2008-03-28 14:52:54,2008-03-28 15:59:59
2,train,2008-03-28 16:00:00,2008-03-28 22:02:00
3,train,2008-03-29 01:27:50,2008-03-29 15:59:59
4,train,2008-03-29 16:00:00,2008-03-30 15:59:59
...,...,...,...
429,taxi,2008-12-07 10:30:54,2008-12-07 10:34:14
430,train,2008-12-07 10:59:29,2008-12-07 11:29:48
431,bus,2008-12-07 11:43:12,2008-12-07 12:23:26
432,walk,2008-12-07 12:23:34,2008-12-07 12:25:07


In [ ]:
p1 p2

p2 p3


----- Trip 1 ^^

p6 p7 

p7 p8 


------- Trip ^^ 2


In [67]:
from tqdm import tqdm

trip_end_row_idxs = list()

for idx in tqdm(range(len(df_labels.index))):
    
    mask = (df_trajectories['DateTime'] >= df_labels.loc[idx, 'StartTime']) & (df_trajectories['DateTime'] <= df_labels.loc[idx, 'EndTime'])
        
    df_trip = df_trajectories[mask]
    trip_indices = df_trip.index
    
    if (len(trip_indices) > 1):
        df_trajectories.loc[trip_indices, 'TransportMode'] = df_labels.loc[idx, 'TransportMode']
        
        if (not df_trip.iloc[0].equals(df_trajectories.iloc[0])):
            if len(trip_end_row_idxs) == 0:
                df_trajectories.drop(list(range(0, trip_indices[0])), inplace=True)
            else:
                df_trajectories.drop(list(range(trip_end_row_idxs[-1] + 1, trip_indices[0])), inplace=True)
    
        trip_end_row_idxs.append(trip_indices[-1])
        
df_trajectories.drop(list(range(trip_end_row_idxs[-1] + 1, df_trajectories.index[-1] + 1)), inplace=True)

100%|██████████| 434/434 [00:38<00:00, 11.38it/s]


In [68]:
df_trajectories

,Latitude,Longitude,Altitude,DateTime,TransportMode
180198,39.894178,116.318200,-777,2008-03-28 14:54:40,train
180199,39.894505,116.321132,-777,2008-03-28 14:55:14,train
180200,39.894953,116.326452,-777,2008-03-28 14:56:13,train
180201,39.894600,116.332542,-777,2008-03-28 14:57:12,train
180202,39.889622,116.337040,-777,2008-03-28 14:58:11,train
...,...,...,...,...,...
720011,39.991644,116.326376,184,2008-12-07 12:37:24,taxi
720012,39.991641,116.326325,180,2008-12-07 12:37:25,taxi
720013,39.991641,116.326289,177,2008-12-07 12:37:26,taxi
720014,39.991643,116.326273,174,2008-12-07 12:37:27,taxi


In [75]:
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert coordinates to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    r = 6371 # radius of the Earth in kilometers
    distance = r * c
    return distance * 1000

In [70]:
df_trajectories_sub = df_trajectories.iloc[1:]

df_trajectories.drop(trip_end_row_idxs[-1], inplace=True)

df_trajectories['LatEnd'] = df_trajectories_sub['Latitude'].tolist()
df_trajectories['LongEnd'] = df_trajectories_sub['Longitude'].tolist()
df_trajectories['AltEnd'] = df_trajectories_sub['Altitude'].tolist()
df_trajectories['TimeEnd'] = df_trajectories_sub['DateTime'].tolist()

df_trajectories.drop(trip_end_row_idxs[:-1], inplace=True)

df_trajectories

,Latitude,Longitude,Altitude,DateTime,TransportMode,LatEnd,LongEnd,AltEnd,TimeEnd
180198,39.894178,116.318200,-777,2008-03-28 14:54:40,train,39.894505,116.321132,-777,2008-03-28 14:55:14
180199,39.894505,116.321132,-777,2008-03-28 14:55:14,train,39.894953,116.326452,-777,2008-03-28 14:56:13
180200,39.894953,116.326452,-777,2008-03-28 14:56:13,train,39.894600,116.332542,-777,2008-03-28 14:57:12
180201,39.894600,116.332542,-777,2008-03-28 14:57:12,train,39.889622,116.337040,-777,2008-03-28 14:58:11
180202,39.889622,116.337040,-777,2008-03-28 14:58:11,train,39.882090,116.338353,-777,2008-03-28 14:59:10
...,...,...,...,...,...,...,...,...,...
720010,39.991633,116.326523,187,2008-12-07 12:37:22,taxi,39.991644,116.326376,184,2008-12-07 12:37:24
720011,39.991644,116.326376,184,2008-12-07 12:37:24,taxi,39.991641,116.326325,180,2008-12-07 12:37:25
720012,39.991641,116.326325,180,2008-12-07 12:37:25,taxi,39.991641,116.326289,177,2008-12-07 12:37:26
720013,39.991641,116.326289,177,2008-12-07 12:37:26,taxi,39.991643,116.326273,174,2008-12-07 12:37:27


In [71]:
df_trajectories['Altitude_avg'] = df_trajectories.apply(lambda row: float(row[2] + row[7]) / 2.0, axis=1)

In [76]:
df_trajectories['Distance(m)'] = df_trajectories.apply(lambda row: haversine_distance(row[0], row[1], row[5], row[6]), axis=1)

df_trajectories

,Latitude,Longitude,Altitude,DateTime,TransportMode,LatEnd,LongEnd,AltEnd,TimeEnd,Altitude_avg,Distance(m)
180198,39.894178,116.318200,-777,2008-03-28 14:54:40,train,39.894505,116.321132,-777,2008-03-28 14:55:14,-777.0,252.763508
180199,39.894505,116.321132,-777,2008-03-28 14:55:14,train,39.894953,116.326452,-777,2008-03-28 14:56:13,-777.0,456.582510
180200,39.894953,116.326452,-777,2008-03-28 14:56:13,train,39.894600,116.332542,-777,2008-03-28 14:57:12,-777.0,521.026904
180201,39.894600,116.332542,-777,2008-03-28 14:57:12,train,39.889622,116.337040,-777,2008-03-28 14:58:11,-777.0,673.538587
180202,39.889622,116.337040,-777,2008-03-28 14:58:11,train,39.882090,116.338353,-777,2008-03-28 14:59:10,-777.0,844.979544
...,...,...,...,...,...,...,...,...,...,...,...
720010,39.991633,116.326523,187,2008-12-07 12:37:22,taxi,39.991644,116.326376,184,2008-12-07 12:37:24,185.5,12.582622
720011,39.991644,116.326376,184,2008-12-07 12:37:24,taxi,39.991641,116.326325,180,2008-12-07 12:37:25,182.0,4.357512
720012,39.991641,116.326325,180,2008-12-07 12:37:25,taxi,39.991641,116.326289,177,2008-12-07 12:37:26,178.5,3.066865
720013,39.991641,116.326289,177,2008-12-07 12:37:26,taxi,39.991643,116.326273,174,2008-12-07 12:37:27,175.5,1.381074


In [78]:
df_trajectories['Duration(s)'] = df_trajectories.apply(lambda row: row[8] - row[3], axis=1)
df_trajectories

,Latitude,Longitude,Altitude,DateTime,TransportMode,LatEnd,LongEnd,AltEnd,TimeEnd,Altitude_avg,Distance(m),Duration(s)
180198,39.894178,116.318200,-777,2008-03-28 14:54:40,train,39.894505,116.321132,-777,2008-03-28 14:55:14,-777.0,252.763508,0 days 00:00:34
180199,39.894505,116.321132,-777,2008-03-28 14:55:14,train,39.894953,116.326452,-777,2008-03-28 14:56:13,-777.0,456.582510,0 days 00:00:59
180200,39.894953,116.326452,-777,2008-03-28 14:56:13,train,39.894600,116.332542,-777,2008-03-28 14:57:12,-777.0,521.026904,0 days 00:00:59
180201,39.894600,116.332542,-777,2008-03-28 14:57:12,train,39.889622,116.337040,-777,2008-03-28 14:58:11,-777.0,673.538587,0 days 00:00:59
180202,39.889622,116.337040,-777,2008-03-28 14:58:11,train,39.882090,116.338353,-777,2008-03-28 14:59:10,-777.0,844.979544,0 days 00:00:59
...,...,...,...,...,...,...,...,...,...,...,...,...
720010,39.991633,116.326523,187,2008-12-07 12:37:22,taxi,39.991644,116.326376,184,2008-12-07 12:37:24,185.5,12.582622,0 days 00:00:02
720011,39.991644,116.326376,184,2008-12-07 12:37:24,taxi,39.991641,116.326325,180,2008-12-07 12:37:25,182.0,4.357512,0 days 00:00:01
720012,39.991641,116.326325,180,2008-12-07 12:37:25,taxi,39.991641,116.326289,177,2008-12-07 12:37:26,178.5,3.066865,0 days 00:00:01
720013,39.991641,116.326289,177,2008-12-07 12:37:26,taxi,39.991643,116.326273,174,2008-12-07 12:37:27,175.5,1.381074,0 days 00:00:01
